In [1]:
import requests
from bs4 import BeautifulSoup
import yfinance as yf
from datetime import datetime
from datetime import timedelta
import pandas as pd
import re
import json
from requests_html import HTMLSession
import nest_asyncio
from dateutil import parser

def changes_from_press(stock_data, press_date, period):
    next_day = timedelta(days=1)
    time_after_release = timedelta(days=period)
    

    next_trading_day = press_date
    
    # try to obtain share price at open on press release date
    day_1_price = stock_data[stock_data['Date'] == press_date]['Open'].values


    # increase the date till the first available open price is found
    num_days = 1
    while not day_1_price:
        if num_days > 30:
            return None
        
        next_trading_day = next_trading_day + next_day

        day_1_price = stock_data[stock_data['Date'] == (next_trading_day)]['Open'].values
        num_days = num_days + 1

    # get next day if available
    next_trading_day = next_trading_day + time_after_release
    day_2_price = stock_data[stock_data['Date'] == (next_trading_day)]['Close'].values

    # increase the date till the second available close price is found
    num_days = 1
    while not day_2_price:
        if num_days > 30: 
            return None
        
        next_trading_day = next_trading_day + next_day

        day_2_price = stock_data[stock_data['Date'] == (next_trading_day + next_day)]['Close'].values
        num_days = num_days + 1
    
    # calcualte percent difference between share prices
    pct_change = ((day_2_price - day_1_price) / day_2_price)*100
    pct_change = pct_change[0]
    return pct_change

def get_df(ticker):
    
    stock_data = yf.Ticker(ticker)

    # get historical market data
    stock_hist = stock_data.history(period="max")

    stock_hist.reset_index(inplace=True)

    stock_hist['Date'] = pd.to_datetime(stock_hist['Date']).dt.date

    stock_hist['Pct_Close'] = stock_hist['Close'].pct_change()*100

    return stock_hist

In [2]:
data = []

ticker = 'WMT'

stock_hist = get_df(ticker)

page_num = 0
while True:
    
    nest_asyncio.apply()

    s = HTMLSession()
    url = f'https://corporate.walmart.com/news/?newsSearchModuleQuery=&0000017d-bf8a-d566-affd-bfba3416000c-page={page_num}'
    r = s.get(url)
    
    soup = BeautifulSoup(r.text) 


    press_statements = soup.find_all('div', {"class": "PromoCardNews"})
    
    if not press_statements:
        break
    
    
    for release in press_statements:
        date = release.find('date', {"class": "Promo-date"}).text.replace('.', '')
        title = release.find('h3', {"class": "PromoCardNews-title"}).text.lstrip().strip()
        
        date = parser.parse(date).date()
        
        pct_change = changes_from_press(stock_hist, date, 1)
        
        if pct_change == None:
            continue
        
        data.append([date, title, pct_change])
        
    page_num = page_num + 1

        


data = pd.DataFrame(data, columns=['date', 'press title', '1d change'])

dates_str = [date.strftime("%m/%d/%Y") for date in data['date'].tolist()]

dict_data = {
    'ticker': ticker,
    
    'data': {
        'dates':dates_str,
        'title': data['press title'].tolist(),
        'priceChange': data['1d change'].tolist(),
    },
}

with open(f'./data/{ticker}.json', 'w', encoding='utf-8') as f:
    json.dump(dict_data, f, ensure_ascii=False, indent=4)



C:\Users\AidanLaptop\AppData\Local\Temp\ipykernel_7632\3079650918.py:41: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  while not day_2_price:


[[datetime.date(2023, 4, 11), 'Walmart Announces Closure of Four Chicago Stores', -1.1115599992071232], [datetime.date(2023, 4, 6), 'Leading the Charge: Walmart Announces Plan To Expand Electric Vehicle Charging Network', 1.1361314978989383], [datetime.date(2023, 4, 5), 'Destination DIY: Walmart Transforms Paint Assortment To Inspire Home Improvements', 2.4071649929387626], [datetime.date(2023, 4, 4), 'Walmart Outlines Growth Strategy, Unveils Next Generation Supply Chain at 2023 Investment Community Meeting', 0.7616752880304193], [datetime.date(2023, 4, 4), 'Walmart and Break the Love Dismiss the Country Club Barrier To Bring the Pickleball Obsession to Everyday Communities', 0.7616752880304193], [datetime.date(2023, 4, 3), 'Walmart Announces April 19 Reopening of Chesapeake Store #1841', -0.33960470998402403], [datetime.date(2023, 4, 3), 'Walmart Turns Up the Excitement and Discovery With Brand-New Walmart.com Experience', -0.33960470998402403], [datetime.date(2023, 3, 30), 'Ozark Tr

[[datetime.date(2023, 4, 11), 'Walmart Announces Closure of Four Chicago Stores', -1.1115599992071232], [datetime.date(2023, 4, 6), 'Leading the Charge: Walmart Announces Plan To Expand Electric Vehicle Charging Network', 1.1361314978989383], [datetime.date(2023, 4, 5), 'Destination DIY: Walmart Transforms Paint Assortment To Inspire Home Improvements', 2.4071649929387626], [datetime.date(2023, 4, 4), 'Walmart Outlines Growth Strategy, Unveils Next Generation Supply Chain at 2023 Investment Community Meeting', 0.7616752880304193], [datetime.date(2023, 4, 4), 'Walmart and Break the Love Dismiss the Country Club Barrier To Bring the Pickleball Obsession to Everyday Communities', 0.7616752880304193], [datetime.date(2023, 4, 3), 'Walmart Announces April 19 Reopening of Chesapeake Store #1841', -0.33960470998402403], [datetime.date(2023, 4, 3), 'Walmart Turns Up the Excitement and Discovery With Brand-New Walmart.com Experience', -0.33960470998402403], [datetime.date(2023, 3, 30), 'Ozark Tr

KeyboardInterrupt: 

In [22]:
press_statements

[]